In [ ]:
import pandas as pd
import requests
import json
import numpy as np
from tabulate import tabulate

def get_and_clean_match_data(hyperlink, season):

  """Fetches match data from FBRef seasons fixture page.
  Cleans it, adds a 'season' column, and returns a pandas DataFrame.

  Args:
    hyperlink: The URL to the FBref fixture page.
    season: A string representing the season (e.g., "2023-2024")."""

  # Fetch the data from the hyperlink
  match_data = pd.read_html(hyperlink)[0]  # First table on page is fixtures table

  # Drop unnecessary columns
  match_data = match_data.drop(columns=['Wk','Day','Time','Attendance','Venue','Referee','Match Report','Notes'])

  # Drop rows where all values are missing ie divider rows
  match_data.dropna(axis=0, how='all', subset=None, inplace=True)

  # Extract home and away scores
  match_data['score_home'] = match_data['Score'].str[0]
  match_data['score_away'] = match_data['Score'].str[2]

  # Convert scores to numeric to enable calculations later
  match_data['score_home'] = pd.to_numeric(match_data['score_home'])
  match_data['score_away'] = pd.to_numeric(match_data['score_away'])

  # Labels rows for season based on 'season' string parameter
  match_data['season'] = season

  return match_data  # Return the cleaned DataFrame


def save_to_json(data, filename):

  """
  Function to save data to JSON file
  Args:
    data: The data to be saved
    filename: The name of the JSON file to save the data to
  """

  if data:  # Only save if data is not empty
      with open(filename, 'w') as file:
          json.dump(data, file, indent=4)
      print(f"Data successfully saved to '{filename}'")
  else:
      print(f"No data to save for '{filename}'")

# Function to fetch data from Odds API
def fetch_odds_data(API_URL, start_date, end_date):

    """
    Function to fetch odds for Odds API. Returns a JSON object.
    Args:
        API_URL: The URL of the Odds API.
        start_date: The start date for Premier League fixtures.
        end_date: The end date for Premier League fixtures.
    """

    API_KEY = "613fd2af3ec961ddefb9b7e0e058bd69"

    start_datetime = start_date+'T00:00:00Z'
    end_datetime = end_date+'T00:00:00Z'

    params = {
        "apiKey": API_KEY,
        "regions": "uk",
        "markets": "h2h",
        "bookmakers": "paddypower",
        "commenceTimeFrom": start_datetime,
        "commenceTimeTo": end_datetime
    }
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        odds_data = response.json()
        save_to_json(odds_data, "match_odds_data.json")
        return odds_data
    else:
        print(f"Error fetching odds data: {response.status_code}")
        return {}

def extract_odds(odds_data):

  """
  Function to turn odds in json format into a dataframe.
  Does not allow overwite if odds alreadty exist for that fixture in the dataframe
  """

  extract_odds = []

  #loop through odds_data
  for match in odds_data:

      #Extract fixture data and teams
      match_data = {
          'Date': pd.to_datetime(match['commence_time']).date(),
          'Home': match['home_team'],
          'Away': match['away_team']
      }

      #Ensure bookmaker dictionary is not empty. This occurs sometimes with Odds API.
      if not match['bookmakers']:
          continue

      #Extract odds for each match
      x = 0
      for outcome in match['bookmakers'][0]['markets'][0]['outcomes']:
          if x == 0:
              match_data['odds_home'] = outcome['price']
          if x == 1:
              match_data['odds_draw'] = outcome['price']
          if x == 2:
              match_data['odds_away'] = outcome['price']
          x += 1

      #add fixture dictionary to list
      extract_odds.append(match_data)

  #Create a dataframe from list of fixtures.
  #Logic to ensure previous odds are not overwritten. This is important because OddsAPI does not provide historical odds (for free). Also odds that update during a game can be highly-skewed, like when a team is losing and game is about to end. So we want to preserve odds acquired before games start.
  try:
      odds_df
  except NameError:  # Use NameError to check for variable existence
      odds_df = pd.DataFrame(extract_odds)
  else:
      extract_df = pd.DataFrame(extract_odds)
      odds_df = pd.concat([odds_df, extract_df], ignore_index=True)
      odds_df = odds_df.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')

  return odds_df

def odds_to_metrics(odds_df, baseline):

  """
  Function enrich odds dataframe with metrics.
  Read more about Poisson distribution and the methodology used here:
  https://www.pinnacle.com/betting-resources/en/soccer/poisson-distribution-predict-the-score-in-soccer-betting/md62mlxumkmxz6a8
  """

  # Step 1: Convert odds to implied probabilities
  odds_df['p_H'] = 1 / odds_df['odds_home']
  odds_df['p_D'] = 1 / odds_df['odds_draw']
  odds_df['p_A'] = 1 / odds_df['odds_away']

  # Step 2: Adjust for the bookmaker’s margin (de‐overround)
  odds_df['S'] = odds_df['p_H'] + odds_df['p_D'] + odds_df['p_A']
  odds_df['P_H'] = odds_df['p_H'] / odds_df['S']
  odds_df['P_A'] = odds_df['p_A'] / odds_df['S']

  # Step 3: Estimate expected goals using a Poisson-inspired adjustment
  # The formula shifts the baseline by half the log ratio of home vs. away win probabilities
  odds_df['odds_home_xG'] = np.round(baseline + 0.5 * np.log(odds_df['P_H'] / odds_df['P_A']),1)
  odds_df['odds_away_xG'] = np.round(baseline - 0.5 * np.log(odds_df['P_H'] / odds_df['P_A']),1)

  # Step 4: Calculate clean sheet likelihoods using the Poisson probability for 0 goals
  # For the home team: the clean sheet likelihood is the probability that the away team scores 0 goals
  odds_df['home_CS%'] = np.round(np.exp(-odds_df['odds_away_xG'])*100,1)
  # For the away team: the clean sheet likelihood is the probability that the home team scores 0 goals
  odds_df['away_CS%'] = np.round(np.exp(-odds_df['odds_home_xG'])*100,1)

  # Drop intermediate columns
  odds_df = odds_df.drop(columns=['p_H', 'p_D', 'p_A', 'S', 'P_H', 'P_A'])

  return odds_df

def calculate_ewma(df, column, span, group_by_col):
    """
    Calculates exponentially weighted moving average (EWMA) for a given column.

    Args:
        df: The DataFrame containing the data.
        column (str): The name of the column to calculate EWMA for.
        span (int): The span for the EWMA calculation ie. the number of historical games to consider.
        group_by_col (str): The column to group by. For this dataset, we are grouping by home and away team.
        adjust: set to True ie. more recent values hold exponentially more weight.

    Returns:
        pd.Series: The EWMA values for the specified column.
    """
    return df.groupby([group_by_col])[column].transform(lambda x: round(x.ewm(span=span, adjust=True).mean(),1))



In [ ]:
#=============================================================
#Get Premier League fixture data
#=============================================================

# Dictionary of FBRef fixtures pages to scrape
season_links = {
    "2024-2025": 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures',
    "2023-2024": 'https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures',
}

# Iterate through the links dict, extract data, and store all season match data in a single dataframe
matches = pd.concat(
    (get_and_clean_match_data(hyperlink, season) for season, hyperlink in season_links.items())
)
# Sort by 'Date' column in ascending order
matches = matches.sort_values(by='Date', ascending=True)

In [ ]:
#=============================================================
#Get odds data + created a dataframe enriched with metrics
#=============================================================

#Get bookmaker odds and turn them into expected goals
ODDS_API_URL = "https://api.the-odds-api.com/v4/sports/soccer_epl/odds"
odds_data = fetch_odds_data(ODDS_API_URL,'2024-08-16','2025-05-25')

# Calculate baseline average expected goals per team
baseline = matches[['score_home', 'score_away']].sum().sum() / len(2*matches)

#Transform odds into expected goals using Poisson distribution
odds_df = odds_to_metrics(extract_odds(odds_data), baseline)



Data successfully saved to 'match_odds_data.json'


In [ ]:
#======================================================
#Create a new merged table. Requires cleaning and renaming.
#Only done for 2024/2025 season. Might have to revisit.
#======================================================

# Make date column a date
matches['Date'] = pd.to_datetime(matches['Date']).dt.date

#make sure 'Home' and 'Away' columns are string with no leading and trailing zeros
matches['Home'] = matches['Home'].astype(str).str.strip()
matches['Away'] = matches['Away'].astype(str).str.strip()
odds_df['Home'] = odds_df['Home'].astype(str).str.strip()
odds_df['Away'] = odds_df['Away'].astype(str).str.strip()

# Update team names in odds table
odds_df.loc[:, ['Home', 'Away']] = odds_df.loc[:, ['Home', 'Away']].replace({
    'Wolverhampton Wanderers': 'Wolves',
    'Brighton and Hove Albion': 'Brighton',
    'West Ham United' : 'West Ham',
    'Tottenham Hotspur': 'Tottenham',
})

# Update team names in matches table
matches.loc[:, ['Home', 'Away']] = matches.loc[:, ['Home', 'Away']].replace({
    'Newcastle Utd': 'Newcastle United',
    "Nott'ham Forest": 'Nottingham Forest',
    'Manchester Utd': 'Manchester United'
})

# Select the desired columns from odds_df
odds_df_subset = odds_df[['Date', 'Home', 'Away', 'odds_home_xG', 'odds_away_xG', 'home_CS%', 'away_CS%']]

# Merge the subset with matches
match_merged = pd.merge(matches, odds_df_subset, on=['Date', 'Home', 'Away'], how='left')

In [ ]:
#==========================================================
#Add new caclulated columns to merged table + export to csv
#==========================================================

# Rename xG columns
match_merged = match_merged.rename(columns={'xG': 'xG_home', 'xG.1': 'xG_away'})

# Calculate moving averages over past 10 games with decay for goals scored and xG
for col, group_col in [('score_home', 'Home'), ('score_away', 'Away'),
                         ('xG_home', 'Home'), ('xG_away', 'Away')]:
    match_merged[col + '_ewma'] = calculate_ewma(match_merged, col, span=10, group_by_col=group_col)

# Calculate diff
match_merged['xG_home_diff'] = np.round(match_merged['score_home'] - match_merged['xG_home'],1)
match_merged['xG_away_diff'] = np.round(match_merged['score_away'] - match_merged['xG_away'],1)
match_merged['ewma_diff_home_xG'] = np.round(match_merged['score_home_ewma'] - match_merged['xG_home_ewma'],1)
match_merged['ewma_diff_away_xG'] = np.round(match_merged['score_away_ewma'] - match_merged['xG_away_ewma'],1)

# Reorder merged table
desired_order = ['season','Date', 'Home', 'Score', 'Away',
                 'home_CS%', 'odds_home_xG' ,'score_home_ewma', 'xG_home_ewma', 'ewma_diff_home_xG',
                 'away_CS%', 'odds_away_xG', 'score_away_ewma', 'xG_away_ewma', 'ewma_diff_away_xG',
                 'score_home','xG_home', 'xG_home_diff',
                 'score_away','xG_away', 'xG_away_diff',
                 ]

match_merged = match_merged[desired_order]

# Export to csv
match_merged.to_csv('EPL_fantasy_metrics.csv', index=False)


In [ ]:
with pd.option_context("display.max_rows", None):
  print(tabulate(match_merged[match_merged['season']=='2024-2025'].head(400), headers='keys', tablefmt='pretty'))

+-----+-----------+------------+-------------------+-------+-------------------+----------+--------------+-----------------+--------------+-------------------+----------+--------------+-----------------+--------------+-------------------+------------+---------+--------------+------------+---------+--------------+
|     |  season   |    Date    |       Home        | Score |       Away        | home_CS% | odds_home_xG | score_home_ewma | xG_home_ewma | ewma_diff_home_xG | away_CS% | odds_away_xG | score_away_ewma | xG_away_ewma | ewma_diff_away_xG | score_home | xG_home | xG_home_diff | score_away | xG_away | xG_away_diff |
+-----+-----------+------------+-------------------+-------+-------------------+----------+--------------+-----------------+--------------+-------------------+----------+--------------+-----------------+--------------+-------------------+------------+---------+--------------+------------+---------+--------------+
| 380 | 2024-2025 | 2024-08-16 | Manchester United |  1

In [ ]:
with pd.option_context("display.max_rows", None):
  print(tabulate(matches[matches['season']=='2024-2025'].head(20), headers='keys', tablefmt='pretty'))

+----+------------+-------------------+-----+-------+------+-------------------+------------+------------+-----------+
|    |    Date    |       Home        | xG  | Score | xG.1 |       Away        | score_home | score_away |  season   |
+----+------------+-------------------+-----+-------+------+-------------------+------------+------------+-----------+
| 0  | 2024-08-16 | Manchester United | 2.4 |  1–0  | 0.4  |      Fulham       |    1.0     |    0.0     | 2024-2025 |
| 1  | 2024-08-17 |   Ipswich Town    | 0.5 |  0–2  | 2.6  |     Liverpool     |    0.0     |    2.0     | 2024-2025 |
| 2  | 2024-08-17 | Newcastle United  | 0.3 |  1–0  | 1.8  |    Southampton    |    1.0     |    0.0     | 2024-2025 |
| 3  | 2024-08-17 | Nottingham Forest | 1.3 |  1–1  | 1.2  |    Bournemouth    |    1.0     |    1.0     | 2024-2025 |
| 4  | 2024-08-17 |      Everton      | 0.5 |  0–3  | 1.4  |     Brighton      |    0.0     |    3.0     | 2024-2025 |
| 5  | 2024-08-17 |      Arsenal      | 1.2 |  2

In [ ]:
with pd.option_context("display.max_rows", None):
  print(tabulate(odds_df.head(20), headers='keys', tablefmt='pretty'))

+----+------------+-------------------+-------------------+-----------+-----------+-----------+--------------+--------------+----------+----------+
|    |    Date    |       Home        |       Away        | odds_home | odds_draw | odds_away | odds_home_xG | odds_away_xG | home_CS% | away_CS% |
+----+------------+-------------------+-------------------+-----------+-----------+-----------+--------------+--------------+----------+----------+
| 0  | 2025-03-16 |      Arsenal      |      Chelsea      |    1.8    |    4.5    |    3.5    |     3.1      |     2.4      |   9.1    |   4.5    |
| 1  | 2025-03-16 |      Fulham       |     Tottenham     |   1.95    |    3.6    |    3.6    |     3.1      |     2.5      |   8.2    |   4.5    |
| 2  | 2025-03-16 |  Leicester City   | Manchester United |    4.2    |   1.83    |    3.6    |     2.7      |     2.8      |   6.1    |   6.7    |
| 3  | 2025-04-01 |      Arsenal      |      Fulham       |   1.36    |    8.0    |    4.5    |     3.4      |  